# Clustering the coworking spaces.

## Merge the description from the JSON file of each city.

In [ ]:
import pandas as pd
import json
import os

# List of JSON file paths (you should replace these with the correct paths to your files)
json_files = [
    "/workspaces/Coworking/src/results/Madrid/Madrid_coworking_spaces.json",
    "/workspaces/Coworking/src/results/Barcelona/Barcelona_coworking_spaces.json",
    "path_to_new_york.json",
    "/workspaces/Coworking/src/results/Tokyo/tokyo_coworking_spaces.json",
    "/workspaces/Coworking/src/results/Sao Paulo/sp_coworking_spaces.json"
]

# List to hold all dataframes
dfs = []

# Load each JSON file and convert it to a DataFrame
for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        dfs.append(df)

# Concatenate all the dataframes into one
merged_df = pd.concat(dfs, ignore_index=True)

# Optional: You can inspect the first few rows
print(merged_df.head())

# Save merged data to CSV
merged_df.to_csv("merged_coworking_spaces.csv", index=False)

# Save merged data to JSON (optional)
merged_df.to_json("merged_coworking_spaces.json", orient="records", lines=True, force_ascii=False)

print("All JSON files have been merged successfully!")


## Using K-Means clustering 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Load the merged DataFrame (assuming it's already loaded in 'merged_df')
# If you haven't loaded it yet, do so:
# merged_df = pd.read_csv("merged_coworking_spaces.csv")

# Clean the 'price' column to extract numerical values
merged_df['price'] = merged_df['price'].str.extract('(\d+)', expand=False)
merged_df['price'] = pd.to_numeric(merged_df['price'], errors='coerce')

# Drop rows where price is missing or invalid
merged_df = merged_df.dropna(subset=['price'])

# Handle text in 'description' column using TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
description_tfidf = tfidf.fit_transform(merged_df['description'])

# Normalize the price data
scaler = StandardScaler()
normalized_price = scaler.fit_transform(merged_df[['price']])

# Convert the sparse matrix from TF-IDF to a dense array
description_tfidf = description_tfidf.toarray()

# Combine the description features (TF-IDF) and price features (normalized)
features = pd.concat([pd.DataFrame(description_tfidf), pd.DataFrame(normalized_price)], axis=1)

# Perform K-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # Set the number of clusters (5 is an example)
merged_df['cluster'] = kmeans.fit_predict(features)

# Visualize the clusters (using the first two features for simplicity)
sns.scatterplot(x=features[:, 0], y=features[:, 1], hue=merged_df['cluster'], palette='viridis')
plt.title('K-Means Clustering of Coworking Spaces')
plt.xlabel('Feature 1 (from description TF-IDF)')
plt.ylabel('Feature 2 (from description TF-IDF)')
plt.legend()
plt.show()

# Optional: Inspect the clusters and the coworking spaces in each cluster
for i in range(5):  # Assuming you have 5 clusters
    print(f"\nCluster {i}:")
    cluster_centers = kmeans.cluster_centers_[i]
    print(f"Cluster center: {cluster_centers}")
    
    cluster_coworking_spaces = merged_df[merged_df['cluster'] == i]
    print(cluster_coworking_spaces[['name', 'price', 'address']].head())
